# 4. 使用 Keras 构建神经网络进行分类 - 印度糖尿病的例子

在这篇文章中，我们将逐步学习使用 keras 库构建神经网络进行分类的方法。

![ls1QvZ](https://upiclw.oss-cn-beijing.aliyuncs.com/uPic/ls1QvZ.jpg)

## 什么是 Keras？

* Keras 是一个用 Python 编写的高级神经网络 API(high-level neural network API)。
* 它能够在 Tensorflow、CNTK 或 Theano 之上(on top of)运行。
* Keras 可以用作深度学习库。支持卷积(Convolutional)和递归神经网络(Recurrent Neural Networks)
* 使用 Keras 进行原型制作(Prototyping)既快速又简单
* 在 CPU 和 GPU 上无缝运行

## 一个简单的二分类神经网络

我们将构建一个用于二进制分类的神经网络。

对于二元分类，我们将使用 [Pima Indians 糖尿病数据库](https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv)进行二元分类。

有 768 个观测值，有 8 个输入变量和 1 个输出变量。

变量名称如下：
* 怀孕(pregnant)的次数。
* 口服葡萄糖耐量试验中 2 小时的血浆葡萄糖浓度/Plasma glucose concentration a 2 hours in an oral glucose tolerance test。
* 舒张压/Diastolic blood pressure  (mm Hg)。
* 三头肌皮褶厚度/Triceps skinfold thickness（mm）。
* 2 小时血清胰岛素/2-Hour serum insulin (mu U/ml)。
* 体重指数/Body mass index（体重公斤/（身高米）²）。
* 糖尿病谱系功能/Diabetes pedigree function。
* 年龄/Age（岁）。
* 类变量/Class variable（0 或 1）。

我们将首先导入基本库 -pandas 和 numpy 以及数据可视化库 matplotlib 和 seaborn。其他库将在使用时导入。

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

我们现在将读取文件并将数据加载到 DataFrame 数据集中

In [1]:
dataset = pd.read_csv('pima_indian_data.csv')

NameError: name 'pd' is not defined

为了更好地理解数据，让我们查看数据集的详细信息。我们需要了解列以及与每列关联的数据类型。

In [ ]:
dataset.head(2)

![uwe7C9](https://upiclw.oss-cn-beijing.aliyuncs.com/uPic/uwe7C9.jpg)

In [ ]:
dataset.describe(include='all')

![EPsJE1](https://upiclw.oss-cn-beijing.aliyuncs.com/uPic/EPsJE1.jpg)

我们可以看到所有特征都是数字的，没有任何分类数据。由于我们没有任何分类变量，我们不需要分类变量的任何数据转换。

我们将数据可视化以便更好地理解。我们使用 **seaborn pairplot** 绘制数据，使用属性 `hue` 将两个类以不同的颜色绘制。

In [ ]:
sns.pairplot(dataset, hue='Class')

![HdE1Qj](https://upiclw.oss-cn-beijing.aliyuncs.com/uPic/HdE1Qj.jpg)

我们使用数据集的相关性绘制热图(heatmap)。这有助于我们消除任何可能无助于预测的特征。

In [ ]:
sns.heatmap(dataset.corr(), annot=True)

![tLZXxl](https://upiclw.oss-cn-beijing.aliyuncs.com/uPic/tLZXxl.jpg)

我们看到所有功能都与 Class 有某种关系，因此我们保留了所有功能。血浆葡萄糖(Plasma glucose)与分类/Class（是否患有糖尿病）的关系最强。年龄(Age)和体重指数(Body Mass Index)也是强大的影响因素。

现在我们了解了数据，让我们创建输入特征和目标变量，并通过预处理数据准备好将其输入到我们的神经网络中的数据。

In [ ]:
# creating input features and target variables
X= dataset.iloc[:,0:8]
y= dataset.iloc[:,8]

让我们看看我们的输入特征

In [ ]:
X.head(2)

![QZogTg](https://upiclw.oss-cn-beijing.aliyuncs.com/uPic/QZogTg.jpg)

由于我们的输入特征处于不同的尺度(different scales)，我们需要对输入进行标准化(standardize)。

In [ ]:
#standardizing the input feature
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)
X


![VOcjLP](https://upiclw.oss-cn-beijing.aliyuncs.com/uPic/VOcjLP.jpg)

Standardized input features

我们现在将输入特征和目标变量拆分为训练数据集和测试数据集。测试数据集将占我们整个数据集的 30%。

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

我们已经对数据进行了预处理，现在可以构建神经网络(neural network)了。

我们正在使用 keras 来构建我们的神经网络。我们导入 keras 库来创建神经网络层。

keras 中有两种主要的模型类型——序列(Sequential)模型和模型。我们将使用序列模型来构建我们的神经网络。

我们使用 **Dense** 库来构建神经网络的输入、隐藏和输出层。

In [ ]:
from keras import Sequential
from keras.layers import Dense

我们有 8 个输入特征(input features)和一个目标变量(one target variable)。 2 隐藏层(Hidden layers)。每个隐藏层将有 4 个节点(nodes)。

**ReLu** 将是隐藏层的激活函数。由于这是一个二元分类问题，我们将使用 **sigmoid** 作为激活函数。

密集层实现

**output = activation(dot(input, kernel) + bias)**

kernel 是权重矩阵。内核/kernel 初始化定义了设置 Keras 层的初始随机权重的方式。

随机正态初始化器生成具有正态分布的张量(Random normal initializer generates tensors with a normal distribution.)。

对于均匀分布，我们可以使用随机均匀初始化器。

Keras 为内核或权重以及偏置单元提供了多个初始化器。

![UZ1yky](https://upiclw.oss-cn-beijing.aliyuncs.com/uPic/UZ1yky.jpg)

In [ ]:
classifier = Sequential()
#First Hidden Layer
classifier.add(Dense(4, activation='relu', kernel_initializer='random_normal', input_dim=8))
#Second  Hidden Layer
classifier.add(Dense(4, activation='relu', kernel_initializer='random_normal'))
#Output Layer
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))

一旦创建了不同的层，我们现在就编译神经网络。

由于这是一个二元分类问题，我们使用 **binary_crossentropy** 来计算实际输出和预测输出之间的损失函数。

为了优化我们的神经网络，我们使用 **Adam**。 **Adam** 代表自适应矩估计(Adaptive moment estimation.)。 **Adam** 是 **RMSProp + Momentum** 的组合。

动量/Momentum 考虑了过去的梯度，以平滑梯度下降。

我们使用 准确性/accuracy 作为衡量模型性能的指标(measure the performance of the model)。

In [ ]:
#Compiling the neural network
classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])

我们现在将训练数据拟合(fit out )到我们创建的模型中。我们使用 10 的 batch_size。这意味着我们每次梯度更新使用 10 个样本。

我们迭代了 100 多个 epoch 来训练模型。一个 epoch 是对整个数据集的迭代。

In [ ]:
#Fitting the data to the training dataset
classifier.fit(X_train,y_train, batch_size=10, epochs=100)

在 100 个 epoch 之后，我们得到大约 80% 的准确率

![VzLBma](https://upiclw.oss-cn-beijing.aliyuncs.com/uPic/VzLBma.jpg)

我们还可以使用评估函数(evaluate function)在测试模式下评估模型的损失值( loss value)和指标值(metrics values for the model).

In [ ]:
eval_model=classifier.evaluate(X_train, y_train)
eval_model

我们现在预测(predict)测试数据集的输出。如果预测值(prediction)大于 0.5，则输出为 1，否则输出为 0

In [ ]:
y_pred=classifier.predict(X_test)
y_pred =(y_pred>0.5)

现在是关键时刻。我们检查测试数据集的准确性

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

![exTbah](https://upiclw.oss-cn-beijing.aliyuncs.com/uPic/exTbah.jpg)

真阳性(true positive)和真阴性( true negative)的总数是测试数据集中 231 个观察值中的 179 个。所以我们对测试数据集的准确率约为 78%。

使用给定的输入，我们可以以 78% 的准确率预测该人是否患有糖尿病